<a href="https://colab.research.google.com/github/noahlearner/autosuggest/blob/main/SEJ_Semantic_Clustering_Tool_by_LeeFootSEO_n_fork_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Semantic Keyword Cluster Tool V1
by [LeeFootSEO](https://twitter.com/LeeFootSEO) February 2022

**Website:** https://www.searchsolved.co.uk

**Twitter:** https://twitter.com/LeeFootSEO

Welcome to Noah's fork of Lee Foot's Amazing Google Colab!!!

Here's how to use it:


1.   Edit the Config Details
2.   Click on Runtime > Run All
2.   The script will install required packages
3.   As the script runs you'll watch each cell run from Top to Bottom.
4.   When the script is done it will diwnload a csv onto your machine.

# Config

In [ ]:
#@markdown ###CSV Output Name 
csv_name = "plumber" #@param {type:"string"} 
#@markdown ###Cluster accuracy  
#@markdown 0-100 (100 = very tight clusters, but higher percentage of no_cluster groups)
cluster_accuracy = 80  #@param {type:"integer"} 
#@markdown ###Minimum Cluster Size 
#@markdown set the minimum size of cluster groups. (Lower number = tighter groups)
min_cluster_size = 2  #@param {type:"integer"} 


# UPLOAD your CSV

*   Script expects a column called 'Keyword' and another called 'Volume'
*   Recommend No More Than 50K Rows

In [ ]:
#@title
from google.colab import files
# upload the keyword export
upload = files.upload()
input_file = list(upload.keys())[0]  # get the name of the uploaded file

In [ ]:
#@title
!pip install sentence_transformers
!pip install pandas
!pip install chardet
!pip install detect_delimiter

In [ ]:
#@title
import sys
import time
import sys
import pandas as pd
import chardet
import codecs
from detect_delimiter import detect
from sentence_transformers import SentenceTransformer, util

# Choose a Sentence Transformer
Download Pre-Trained Models: https://www.sbert.net/docs/pretrained_models.html

In [ ]:
#@title
transformer = 'all-mpnet-base-v2'  # provides the best quality
#transformer = 'all-MiniLM-L6-v2'  # 5 times faster and still offers good quality

In [ ]:
#@title
# automatically detect the character encoding type

acceptable_confidence = .8

contents = upload[input_file]

codec_enc_mapping = {
    codecs.BOM_UTF8: 'utf-8-sig',
    codecs.BOM_UTF16: 'utf-16',
    codecs.BOM_UTF16_BE: 'utf-16-be',
    codecs.BOM_UTF16_LE: 'utf-16-le',
    codecs.BOM_UTF32: 'utf-32',
    codecs.BOM_UTF32_BE: 'utf-32-be',
    codecs.BOM_UTF32_LE: 'utf-32-le',
}

encoding_type = 'utf-8'  # Default assumption
is_unicode = False

for bom, enc in codec_enc_mapping.items():
    if contents.startswith(bom):
        encoding_type = enc
        is_unicode = True
        break

if not is_unicode:
    # Didn't find BOM, so let's try to detect the encoding
    guess = chardet.detect(contents)
    if guess['confidence'] >= acceptable_confidence:
        encoding_type = guess['encoding']

print("Character Encoding Type Detected", encoding_type)

In [ ]:
#@title
# automatically detect the delimiter
with open(input_file,encoding=encoding_type) as myfile:
    firstline = myfile.readline()
myfile.close()
delimiter_type = detect(firstline)

In [ ]:
#@title
# create a dataframe using the detected delimiter and encoding type
df = pd.read_csv((input_file), on_bad_lines='skip', encoding=encoding_type, delimiter=delimiter_type)
count_rows = len(df)
if count_rows > 50_000:
  print("WARNING: You May Experience Crashes When Processing Over 50,000 Keywords at Once. Please consider smaller batches!")
print("Uploaded Keyword CSV File Successfully!")
df

In [ ]:
#@title
# standardise the keyword columns
df.rename(columns={"Search term": "Keyword", "keyword": "Keyword", "query": "Keyword", "query": "Keyword", "Top queries": "Keyword", "queries": "Keyword", "Keywords": "Keyword","keywords": "Keyword", "Search terms report": "Keyword"}, inplace=True)

if "Keyword" not in df.columns:
  print("Error! Please make sure your csv file contains a column named 'Keyword!")

# store the data
cluster_name_list = []
corpus_sentences_list = []
df_all = []

corpus_set = set(df['Keyword'])
corpus_set_all = corpus_set
cluster = True


#Clustering Keywords - This can take a while!

In [ ]:
#@title
# keep looping through until no more clusters are created

cluster_accuracy = cluster_accuracy / 100
model = SentenceTransformer(transformer)

while cluster:

    corpus_sentences = list(corpus_set)
    check_len = len(corpus_sentences)

    corpus_embeddings = model.encode(corpus_sentences, batch_size=256, show_progress_bar=True, convert_to_tensor=True)
    clusters = util.community_detection(corpus_embeddings, min_community_size=min_cluster_size, threshold=cluster_accuracy, init_max_size=len(corpus_embeddings))

    for keyword, cluster in enumerate(clusters):
        print("\nCluster {}, #{} Elements ".format(keyword + 1, len(cluster)))

        for sentence_id in cluster[0:]:
            print("\t", corpus_sentences[sentence_id])
            corpus_sentences_list.append(corpus_sentences[sentence_id])
            cluster_name_list.append("Cluster {}, #{} Elements ".format(keyword + 1, len(cluster)))

    df_new = pd.DataFrame(None)
    df_new['Cluster Name'] = cluster_name_list
    df_new["Keyword"] = corpus_sentences_list


    df_all.append(df_new)
    have = set(df_new["Keyword"])

    corpus_set = corpus_set_all - have
    remaining = len(corpus_set)
    print("Total Unclustered Keywords: ", remaining)
    if check_len == remaining:
        break

In [ ]:
#@title
# make a new dataframe from the list of dataframe and merge back into the orginal df
df_new = pd.concat(df_all)
df = df.merge(df_new.drop_duplicates('Keyword'), how='left', on="Keyword")

In [ ]:
#@title

# rename the clusters to the shortest keyword in the cluster
df['Length'] = df['Keyword'].astype(str).map(len)
df['Volume'] = df['Volume'].astype(int)
df = df.sort_values(by="Volume", ascending=False)

df['Cluster Name'] = df.groupby('Cluster Name')['Keyword'].transform('first')
df.sort_values(['Cluster Name', "Keyword"], ascending=[True, True], inplace=True)

df['Cluster Name'] = df['Cluster Name'].fillna("zzz_no_cluster")

del df['Length']

In [ ]:
#@title
# move the cluster and keyword columns to the front
col = df.pop("Keyword")
df.insert(0, col.name, col)

col = df.pop('Cluster Name')
df.insert(0, col.name, col)

df.sort_values(["Cluster Name", "Volume"], ascending=[True, False], inplace=True)

In [ ]:
#@title
uncluster_percent = (remaining / count_rows) * 100
clustered_percent = 100 - uncluster_percent
print(clustered_percent,"% of rows clustered successfully!")

In [ ]:
#@title
df.to_csv(csv_name, index=False)
files.download(csv_name + ".csv")